# Connexion

[Connexion](https://github.com/zalando/connexion) is a python framework based on flask.

It simplifies greatly the creation of REST APIs.

Once you have your OAS3 spec, `connexion` can use it to:

- dispatch requests
- validate input and output of the called methods
- apply authentication policies

In [1]:
# At first ensure connexion is installed 
# together with the swagger module used to render the OAS3 spec
# in the web-ui
!pip install connexion[swagger_ui] connexion

Now  [run the spec in a terminal](/terminals/1) using

```
connexion run /code/notebooks/oas3/ex-1-info-ok.yaml
```

and play a bit with the UI

## Defining endpoints in OAS3

Now that we have added our metadata, we can provide the informations about endpoints.
OAS3 allows multiple endpoint because every good API has a test / sandbox infrastructure.

```
# One or more server
#   You can add production, staging and test environments.
#   We are using the custom `x-sandbox` to identify
#   sandbox instances
servers:
  - description: Development server
    url: https://localhost:8443/datetime/v1
  - description: Test server
    url: https://api.example.com/datetime/v1
    x-sandbox: true
    # The following custom parameter can be used
    # to declare how to check the API.
    x-healthCheck:
      url: https://api.example.com/datetime/v1/status
      interval: 300
      timeout: 15

```

### Exercise

Edit the servers tag so that it points to your actual endpoint URL (eg. your IP/port).

## Adding a path

Now we can define our first `path` that is the `/status` one.
An interoperable API should have a well known url usable for testing its status, so that
the implementers can decide which is a suitable method for testing it (eg. it could be
a simple OK/KO method or can execute basic checks (eg. databases are reachable, smoke testing other components, ..)

NB: the `/status` path is not a replacement for proper monitoring your APIs, but a way to communicate to your peers that you're online.

An OAS3 path references:
    
    - the associated METHOD
    - a `summary` and a `description` of the method


```  
  /status:
    get:
      summary: Ritorna lo stato dell'applicazione.
      description: |
        Ritorna lo stato dell'applicazione. A scopo
        di test, su base randomica puo' ritornare
        un errore.
```
    - a reference to the python object to call when the 
   
```
      operationId: get_status
```

    - the http statuses of the possible responses, each with its description,
      content-type and examples
      
```
      responses:
        '200':
          description: |
            Il server ha ritornato lo status. In caso di problemi
            ritorna sempre un problem+json.
          content:
            application/problem+json:
              example:
                status: 200
                title: OK
                detail: API is working properly.
        default:
          description: |
            If none of the above statuses is returned, then this applies
          content:
            application/problem+json:
              example:
                status: 500
                title: Internal Server Error
                detail: API is not responding correctly
            
           

```

## Exercise

We haven't already implemented the operationId
`get_status()`, so  [to run the spec in a terminal](/terminals/1) we should tell the server
to ignore this.

```
connexion run /code/notebooks/oas3/ex-03-02-path.yaml --stub
```

## Exercise

1- What happens if I get the `/status` resource of my API now?

2- And if I invoke another path which is not mentioned in the spec?

3- Restart the server via

```
connexion ru
```

Solution on the unimplemented method

```
$ curl http://0.0.0.0:8889/datetime/v1/status
{
  "detail": "Empty module name",
  "status": 501,
  "title": "Not Implemented",
  "type": "about:blank"
}
```

Solution on other paths

```
$ curl http://0.0.0.0:8889/datetime/v1/missing
{
  "detail": "The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.",
  "status": 404,
  "title": "Not Found",
  "type": "about:blank"
}
```

## Schemas

OAS3 allows defining, using and reusing schemas. They can be defined inline,  in the `component` section or referenced from another file, like below.

```
components:
    schemas:
      Problem:
        $ref: 'https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml#/schemas/Problem'

```


## Exercise

Edit /code/notebooks/oas3/ex-03-02-path.yaml so that every `/status` response uses
the `Problem` schema.

Look at [simple.yaml](https://github.com/teamdigitale/api-starter-kit/blob/master/openapi/simple.yaml) to
see a complete implementation.

In [ ]:
## Exercise

